In [1]:
import duckdb as ddb
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.gridspec as gridspec

data = ddb.read_parquet("all_reviews/all_reviews.parquet")

In [ ]:
# columns and types
for x, item in enumerate(data.columns):
    print(str(item) + ":  " + str(data.dtypes[x]))

game:  VARCHAR
author_steamid:  BIGINT
author_num_games_owned:  INTEGER
author_num_reviews:  INTEGER
author_playtime_forever:  INTEGER
author_playtime_last_two_weeks:  INTEGER
author_playtime_at_review:  BIGINT
author_last_played:  INTEGER
language:  VARCHAR
review:  VARCHAR
timestamp_created:  INTEGER
timestamp_updated:  INTEGER
voted_up:  INTEGER
votes_up:  INTEGER
votes_funny:  INTEGER
weighted_vote_score:  FLOAT
comment_count:  INTEGER
steam_purchase:  INTEGER
received_for_free:  INTEGER
written_during_early_access:  INTEGER


In [ ]:
# print(data.describe())


['game', 'author_steamid', 'author_num_games_owned', 'author_num_reviews', 'author_playtime_forever', 'author_playtime_last_two_weeks', 'author_playtime_at_review', 'author_last_played', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access']
┌─────────┬─────────────────────────────┬───────────────────────┬────────────────────────┬────────────────────┬─────────────────────────┬────────────────────────────────┬───────────────────────────┬────────────────────┬────────────┬───────────────────────────────────────────────┬────────────────────┬────────────────────┬───────────────────┬────────────────────┬────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────────────┐
│  aggr   │            game             │    author_steamid     │ author_num_games_owned │ author

In [ ]:
# IDEA:
# review scores from those who got the game for free vs those who had to pay for it
# is there an average of a higher rating for free / not purchased? for what games?

# plan: 
#   gather the top 3 games the most amount of reviews
#   find out the percentage of positive vs negative reviews
#   then find out the percentages from those who puchased vs did not purchase the game (include those who got the game for free)
#   analyze which games had which results, and do some further research on why this may be
#       this could be finding the reviews with the highest helpfulness or most upvotes
#       this could also include doing some outside reseach on reddit / internet

# gather the top 3 games the most amount of reviews
most_reviewed_games = ddb.sql(f"""
    SELECT game, COUNT(*) as review_count
    FROM data
    GROUP BY game
    ORDER BY review_count DESC
    LIMIT 3
""").fetchdf()

print(most_reviewed_games)

                 game  review_count
0  Grand Theft Auto V       1659263
1            Terraria       1205564
2                Rust        974388


In [ ]:
# find out the percentage of positive vs negative reviews
# some columns to consider:
#   voted_up:  INTEGER
top_game_review_percentages = ddb.sql(f"""
    SELECT
        game,
        SUM(CASE WHEN voted_up = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS positive_review_percentage
    FROM data
    WHERE game IN (SELECT game FROM most_reviewed_games)
    GROUP BY game
    ORDER BY positive_review_percentage DESC;
""").fetchdf()

print(top_game_review_percentages)